In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [2]:
# Model Parameter
model = "resnet34"
model_short = "resnet"
MODEL_MODULE = resnet34
# Dataset parameter
dataset = "tinyimagenet"
dataset_short = "tinyimage"
DATASET_ENUM = DatasetClassN.Tinyimage

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 5
MULTI_WORKERS_N = 4
STANDARDIZE = False

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [3]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [4]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [5]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, data_class=DATASET_ENUM, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH, standardize=STANDARDIZE)

NON Members
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 25000
Batchsize: 1
Inputs:tensor([[[[0.5176, 0.5373, 0.5098,  ..., 0.4196, 0.4196, 0.4196], [0.6157, 0.6000, 0.5647,  ..., 0.4118, 0.4157, 0.4157], [0.5961, 0.5647, 0.5294,  ..., 0.4000, 0.4039, 0.4039], ..., [0.1765, 0.1216, 0.0980,  ..., 0.2588, 0.2392, 0.2118], [0.0471, 0.1804, 0.1647,  ..., 0.2902, 0.2667, 0.2275], [0.1725, 0.1255, 0.0745,  ..., 0.2980, 0.2667, 0.2392]],[[0.5216, 0.5412, 0.5137,  ..., 0.3882, 0.3882, 0.3882], [0.6196, 0.6039, 0.5686,  ..., 0.3804, 0.3843, 0.3843], [0.6000, 0.5686, 0.5333,  ..., 0.3686, 0.3725, 0.3725], ..., [0.1765, 0.1216, 0.0863,  ..., 0.1804, 0.1529, 0.1333], [0.0353, 0.1647, 0.1373,  ..., 0.2039, 0.1725, 0.1412], [0.1569, 0.0980, 0.0392,  ..., 0.2039, 0.1725, 0.1451]],[[0.4980, 0.5176, 0.4902,  ..., 0.2980, 0.2980, 0.2980], [0.5961, 0.5804, 0.5451,  ..., 0.2902, 0.2941, 0.2941], [0.5804, 0.5490, 0.5137,  ..., 0.2863, 0.2902, 0.2902], ..., [0

In [6]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 0.3576,  5.9040,  5.2336,  ...,  0.0000,  0.0000,  0.0000]],  [[-1.1405,  3.3597,  3.3209,  ...,  0.0000,  0.0000,  0.0000]],  [[-1.5036,  2.8377,  2.6603,  ...,  0.0000,  0.0000,  0.0000]],  ...,  [[ 1.9527,  6.9611,  6.7171,  ...,  0.0000,  0.0000,  0.0000]],  [[-1.1390,  3.2491,  2.7955,  ...,  0.0000,  0.0000,  0.0000]],  [[ 3.4227,  8.2669,  7.7257,  ...,  0.0000,  0.0000,  0.0000]]]), tensor([1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,  0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,  0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1])]


### Attack Model

Simple Model

In [7]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [8]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(203, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.fc5 = nn.Linear(32, 16)
        self.bn5 = nn.BatchNorm1d(16)
        self.fc6 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [9]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [11]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, data_class=DATASET_ENUM, device=DEVICE, test_dataset=False, standardize=STANDARDIZE)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.3490, 0.7451, 0.2863,  ..., 0.6275, 0.6235, 0.6157], [0.7647, 0.7569, 0.6118,  ..., 0.6196, 0.5804, 0.5882], [0.6824, 0.5569, 0.6980,  ..., 0.6588, 0.6627, 0.6863], ..., [0.6431, 0.6588, 0.5765,  ..., 0.6863, 0.5961, 0.4078], [0.7490, 0.7255, 0.7020,  ..., 0.6000, 0.5804, 0.7961], [0.7098, 0.7333, 0.6941,  ..., 0.5686, 0.7098, 0.5843]],[[0.3490, 0.7451, 0.2863,  ..., 0.6118, 0.6078, 0.6000], [0.7647, 0.7569, 0.6118,  ..., 0.6039, 0.5647, 0.5725], [0.6824, 0.5569, 0.6980,  ..., 0.6431, 0.6353, 0.6588], ..., [0.6471, 0.6627, 0.5804,  ..., 0.6902, 0.6000, 0.4118], [0.7529, 0.7294, 0.7059,  ..., 0.6039, 0.5843, 0.8000], [0.7137, 0.7373, 0.6980,  ..., 0.5725, 0.7137, 0.5882]],[[0.3098, 0.7059, 0.2471,  ..., 0.5765, 0.5647, 0.5569], [0.7255, 0.7176, 0.5725,  ..., 0.5686, 0.5216, 0.5294], [0.6510, 0.5255, 0.6667,  ..., 0.6000, 0.5961, 0.6196], ..., [0.6235, 0.6392,

Attack Evaluation DL Sample

In [12]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[0.9520, 2.7617, 2.5413,  ..., 0.0000, 0.0000, 0.0000]],  [[0.8969, 3.2163, 2.8674,  ..., 0.0000, 0.0000, 0.0000]],  [[1.0188, 3.8211, 3.8029,  ..., 0.0000, 0.0000, 0.0000]],  ...,  [[1.0011, 3.2814, 3.1813,  ..., 0.0000, 0.0000, 0.0000]],  [[0.8768, 2.6851, 2.6250,  ..., 0.0000, 0.0000, 0.0000]],  [[0.9756, 3.3248, 3.3173,  ..., 0.0000, 0.0000, 0.0000]]]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]


### Train Attack model

In [13]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 50000
Batchsize: 64
Inputs:tensor([[[ 0.3307,  5.3064,  4.3894,  ...,  0.0000,  0.0000,  0.0000]],  [[ 1.3572,  9.2462,  7.6549,  ...,  0.0000,  0.0000,  0.0000]],  [[-0.6161,  4.7033,  3.9173,  ...,  0.0000,  0.0000,  0.0000]],  ...,  [[-0.3023,  5.6641,  5.1698,  ...,  0.0000,  0.0000,  0.0000]],  [[-0.6815,  4.3297,  3.9437,  ...,  0.0000,  0.0000,  0.0000]],  [[-0.5471,  3.8861,  3.8354,  ...,  0.0000,  0.0000,  0.0000]]])
Squeezed Input: tensor([[ 0.3307,  5.3064,  4.3894,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.3572,  9.2462,  7.6549,  ...,  0.0000,  0.0000,  0.0000],
        [-0.6161,  4.7033,  3.9173,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.3023,  5.6641,  5.1698,  ...,  0.0000,  0.0000,  0.0000],
        [-0.6815,  4.3297,  3.9437,  ...,  0.0000,  0.0000,  0.0000],
        [-0.5471,  3.8861,  3.8354,  ...,  0.0000,  0.0000,  0.0000]])
Labels:tensor([[0

### Evaluation

In [14]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 0.52 with Correct: 103 and Total: 200


0.515